In [33]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [15]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

In [4]:
 ! pip install pytest-playwright
 ! playwright install

In [1]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

In [ ]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

✅ Intercepted post 3583382089212371814_54284618


In [5]:
! pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [7]:
from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [8]:
username

'r31412718'

In [ ]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        
        await page.wait_for_selector('input[name="username"]', timeout=10000)
        await asyncio.sleep(0.5)
        await page.fill('input[name="username"]', username)
        await asyncio.sleep(0.5)
        await page.fill('input[name="password"]', password)
        await asyncio.sleep(0.5)

        await page.click('button[type="submit"]')
        await asyncio.sleep(1)

        # Wait for navigation or check for login errors
        await page.wait_for_selector('nav', timeout=10000)  # Wait for main navigation bar as successful login indicator

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

In [31]:
await signon(page, username, password)

ValueError: Login failed: Incorrect username or password.

In [32]:
async def scrape_profile_posts(profile_url: str, page):
    target_endpoint_pattern = r"/api/v1/media/\d+/info/"
    intercepted_posts = {}
    clicked_hrefs = set()

    # await signon(page, username, password)

    async def intercept_response(response):
        if re.search(target_endpoint_pattern, response.url) and response.status == 200:
            data = await response.json()
            media_id = data.get('items', [{}])[0].get('id', 'unknown')
            intercepted_posts[media_id] = data
            print(f"✅ Intercepted post {media_id}")

    page.on("response", intercept_response)

    print(f"Navigating to profile: {profile_url}")
    
    await page.goto(profile_url, wait_until="networkidle")

    # Click on the "feed" button if necessary (replace with actual selector)
    # Example: await page.click('selector-for-feed-button')

    posts = await page.query_selector_all("div._aagu")
    print(f"Found {len(posts)} potential posts.")

    for post in posts:
        parent = await post.evaluate_handle("node => node.closest('a')")
        if parent:
            href = await parent.get_attribute('href')
            if href and '/p/' in href and href not in clicked_hrefs:
                print(f"Clicking on post: {href}")
                clicked_hrefs.add(href)
                future_response = asyncio.Future()

                async def response_listener(response):
                    if re.search(target_endpoint_pattern, response.url) and response.status == 200:
                        if not future_response.done():
                            future_response.set_result(True)

                page.on("response", response_listener)
                await parent.click()

                try:
                    await asyncio.wait_for(future_response, timeout=10)
                    print(f"Data intercepted successfully for {href}")
                except asyncio.TimeoutError:
                    print(f"Failed to intercept data for {href}")

                await asyncio.sleep(1)  # Adjust sleep if needed
                await page.go_back(wait_until="networkidle")

    print("Completed scraping posts.")
    return intercepted_posts

# Usage:
# Assuming PAGE is already initialized Playwright page
data = await scrape_profile_posts("https://www.instagram.com/mattrife/", page)
print(json.dumps(data, indent=2))


Navigating to profile: https://www.instagram.com/mattrife/
Found 24 potential posts.
Clicking on post: /mattrife/p/DG6uwZKACNm/
Data intercepted successfully for /mattrife/p/DG6uwZKACNm/


Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - navigating to "https://www.instagram.com/mattrife/", waiting until "networkidle"\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - navigating to "https://www.instagram.com/mattrife/", waiting until "networkidle"



Clicking on post: /mattrife/p/DG1rLC4gVnM/


Error: ElementHandle.click: Element is not attached to the DOM
Call log:
  - attempting click action
  -     - waiting for element to be visible, enabled and stable
